In [1]:
# Refresh README.md (Week1 & Week2 Sections 1..5)
from __future__ import annotations
from pathlib import Path
from datetime import datetime
import argparse

TITLES = {
    # Week 2
    "Wk02_Section1": "Time-Based Feature Engineering",
    "Wk02_Section2": "Lags & Rolling Statistics",
    "Wk02_Section3": "Feature Scaling & Normalization",
    "Wk02_Section4": "Data Splitting & Preparation",
    "Wk02_Section5": "Data Quality & Preprocessing",
    # Week 1 (update if you have exact titles)
    "Wk01_Section1": "Time Checks & Overview",
    "Wk01_Section2": "Week 1 — Section 2",
    "Wk01_Section3": "Week 1 — Section 3",
    "Wk01_Section4": "Week 1 — Section 4",
    "Wk01_Section5": "Week 1 — Section 5",
}

def find_base_dir(start: Path) -> Path:
    p = start.resolve()
    for _ in range(8):
        if (p/"Code").exists() and ((p/"results").exists() or (p/"data").exists()):
            return p
        if p.name.lower()=="powercast" and (p/"Code").exists():
            return p
        p = p.parent
    return start.resolve()

def collect_sections(results_dir: Path, week_prefix: str, repo_prefix: str) -> list[str]:
    blocks = []
    for sec in range(1, 6):
        sec_key = f"{week_prefix}_Section{sec}"
        sec_dir = results_dir / sec_key
        if not sec_dir.exists(): 
            continue

        plots_dir = sec_dir / "plots"
        reports_dir = sec_dir / "reports"
        title = TITLES.get(sec_key, sec_key)

        lines = [f"### {sec_key} — {title}"]

        if plots_dir.exists():
            for png in sorted(plots_dir.glob("*.png")):
                rel = f"{repo_prefix}results/{sec_key}/plots/{png.name}"
                friendly = png.stem.replace("_", " ").title()
                lines.append(f"- 📊 [{friendly}]({rel})")

        if reports_dir.exists():
            report = reports_dir / f"SDS-CP036-powercast_{sec_key}_Business_Report.md"
            if report.exists():
                rel = f"{repo_prefix}results/{sec_key}/reports/{report.name}"
                lines.append(f"- 💼 [Business Report]({rel})")

        blocks.append("\n".join(lines))
    return blocks

def build_readme(base: Path, repo_prefix: str="./") -> Path:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    results = base / "results"

    wk2_blocks = collect_sections(results, "Wk02", repo_prefix)
    wk1_blocks = collect_sections(results, "Wk01", repo_prefix)

    consolidated = []
    wk2 = base / "SDS-CP036-powercast_Wk02_Report_Business.md"
    if wk2.exists():
        consolidated.append(f"- 📘 [Week 2 Consolidated Business Report]({repo_prefix}{wk2.name})")
    wk1 = base / "SDS-CP036-powercast_Wk01_Report_Business.md"
    if wk1.exists():
        consolidated.append(f"- 📘 [Week 1 Consolidated Business Report]({repo_prefix}{wk1.name})")

    lines = [
        "# SDS-CP036-powercast — Results Overview",
        "",
        f"📅 **Generated:** {ts}",
        "",
        "---",
        "",
        "## Week 2 — Section Reports",
        "",
        *(wk2_blocks if wk2_blocks else ["_No Week 2 outputs found._"]),
        "",
        "---",
        "",
        "## Week 1 — Section Reports",
        "",
        *(wk1_blocks if wk1_blocks else ["_No Week 1 outputs found._"]),
        "",
        "---",
        "",
        "## Consolidated Reports",
        "",
        *(consolidated if consolidated else ["_No consolidated reports present._"]),
        "",
    ]

    out = base / "README.md"
    out.write_text("\n".join(lines), encoding="utf-8")
    return out

# ---- Execute with local relative links by default ----
BASE = find_base_dir(Path.cwd())
path = build_readme(BASE, repo_prefix="./")
print(f"README regenerated at: {path}")


README regenerated at: /home/6376f5a9-d12b-4255-9426-c0091ad440a7/Powercast/README.md
